<a href="https://colab.research.google.com/github/zehrahayirci/instagram_t-SNE/blob/master/insta_t_SNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello! In this Colab, we mplement t-SNE algorithm for color histagram and ImageNet classes of Instagram posts

*   List item
*   List item



In [0]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import Image
import glob
import pickle
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist

First, we download photos from instagram by [instaloader](https://pypi.org/project/instaloader/)



---



In [0]:
!pip install instaloader
profilename = 'tokyoda100gun' #that's me!

We will get all photos of one user with
`instaloader profile [profile ...]`

Keep in mind that, this colab works best for 100+ posts to create 10x10 grid image


In [0]:
!instaloader profile tokyoda100gun 

If your profile is more than 100, or you want to add/discard manually you can use the Files bar on the left and locate your profile name. 
I want to use resized images. 


In [0]:

instagram_files = "/content/" + profilename + "/*.jpg"
resize_files = "/content/" + profilename +"/"
for f,insta_file in enumerate(glob.glob(instagram_files)):
    image = Image.open(insta_file)
    resized_img =  image.resize((100,100), Image.ANTIALIAS)

    #resized_img = Image.fromarray(resized_img)
    outfile = resize_files + insta_file
    print(resize_files+str(f)+'.png')
    resized_img.save(resize_files+str(f)+'.png', 'png', quality=80)
    if f > 98:
      break
    


First, we create histograms of the posts. Sorry for all b&w posts!

In [0]:
resize_files = "/content/" + profilename +"/*.png"
color = ('b','g','r')
files = [resize_files]
hist = {}
index = 0 
for filem in files:
	hist[filem]={}
	for filename in glob.glob(filem):
		hist[filem][filename]={}
		#print(filename)
		for i,col in enumerate(color):
			im=Image.open(filename)
			image_data = np.asarray(im)
			if len(image_data.shape) == 2 :
				image_data = cv.cvtColor(image_data,cv.COLOR_GRAY2RGB)
			index = index +1
			hist[filem][filename][col] = cv.calcHist([image_data],[i],None,[256],[0,256])

with open('insta_histogram.pickle', 'wb') as f:
	pickle.dump(hist, f)
print(index) #this should return 3 x your post count 


Create a list of all histogram of all images. 
I set perplexity to 25, but you can play it as you want

In [0]:
with open('insta_histogram.pickle', 'rb') as f:
	data = pickle.load(f)
hist = np.zeros((100,768))
filenames = []
for fil, filem in enumerate(data): #1
	for fn,filename in enumerate(data[filem]): #400
		hist[fn,0:256] = data[filem][filename]['b'][:,0] #256
		hist[fn,256:512] = data[filem][filename]['g'][:,0] #256
		hist[fn,512:768] = data[filem][filename]['r'][:,0] #256
		filenames.append((filem,filename))

		
tsne = TSNE(perplexity=25, n_components=2, init='random', n_iter=5000)
X_2d = tsne.fit_transform(np.array(hist))
X_2d -= X_2d.min(axis=0)
X_2d /= X_2d.max(axis=0)

with open('tsne.pickle', 'wb') as f:
	pickle.dump({'tsne':X_2d,'name':filenames}, f)

We need lapjv to create a nice grid from t-SNE results

In [0]:
!pip install lapjv
from lapjv import lapjv

In [0]:
grid = np.dstack(np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))).reshape(-1, 2)
cost_matrix = cdist(grid, X_2d, "sqeuclidean").astype(np.float32)
out_res = 120

In [0]:
cost_matrix = cost_matrix * (100000 / cost_matrix.max())
row_asses, col_asses, _ = lapjv(cost_matrix)
grid_jv = grid[col_asses]

In [0]:
out = np.ones((10*out_res, 10*out_res, 3))

for i in range(100):
	coordinate_X =  int(np.floor(grid_jv[i,1]*9*out_res))
	coordinate_Y =  int(np.floor(grid_jv[i,0]*9*out_res))

	im=Image.open(filenames[i][1]).convert('RGB').resize((120,120))
	image_data = np.asarray(im)
	out[coordinate_Y:coordinate_Y + out_res, coordinate_X:coordinate_X + out_res]  = image_data


In [0]:
im = Image.fromarray(out.astype(np.uint8))
im.save("insta_histogram.png")
plt.imshow(im)

Perfect! now, we can continue to ImageNet classes and create another cost funtion.

I use Resnet for it 

In [0]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
resnet18.eval()

I don't want to results for Imagenet, instead, I will use the last features.

I discard the last activation step from the network.

Please enable GPU from Edit -> Notebook Settings -> Hardware

In [0]:
import torch.nn as nn
modus=list(resnet18.children())[:-1]
resnet18_feature=nn.Sequential(*modus)
resnet18_feature.cuda()
for p in resnet18_feature.parameters():
  p.requires_grad = False

Again we will use 100 images. Go to the link below and allw Google Drive acces by copy paste the code.

In [0]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
files = [resize_files]
features = {}
features[folder] = {}
for filem in files:
  for img in  glob.glob(filem):

    insta_image = Image.open(img)
    print(img)
    img_t = transform(insta_image)
    batch_t = torch.unsqueeze(img_t, 0)

    out_feature = resnet18_feature(batch_t.cuda())
    out_feature = out_feature.view(512)
    features[folder][img] = out_feature.cpu().detach().numpy().reshape(512)

    



In [0]:
features = {}
features[folder] = {}
print(all_images)
for filem in all_images:
  for img in  glob.glob(filem):

    insta_image = Image.open(img)
    #If your images include b&w, convert to RGB format uncomment below
    #insta_image = Image.open(insta_image).convert('RGB')

    img_t = transform(insta_image)
    batch_t = torch.unsqueeze(img_t, 0)

    #hangi_class = np.argmax(out_class.cpu().detach().numpy())
    #print(hangi_class)
    #plt.plot(out.cpu().detach().numpy().reshape(1000))
    out_feature = resnet18_feature(batch_t.cuda())
    out_feature = out_feature.view(512)
    features[img] = out_feature.cpu().detach().numpy().reshape(512)
    #plt.plot(out.cpu().detach().numpy().reshape(512))
    

In [0]:
with open('features.pickle', 'wb') as f:
	pickle.dump(features, f)
  #features.keys()

Now we have all feature in the pickle file, we can visualize them just like histogram.

In [0]:
with open('features.pickle', 'rb') as g:
	features = pickle.load(g)
  
feat = np.zeros((100,512))

for fil, filem in enumerate(features): #1
  for fn,filename in enumerate(features[filem]):
    feat[fil]=features[filem][filename]


In [0]:

tsne = TSNE(perplexity=25, n_components=2, init='random', n_iter=5000)
X_2d = tsne.fit_transform(np.array(feat))
X_2d -= X_2d.min(axis=0)
X_2d /= X_2d.max(axis=0)
with open('tsne_features.pickle', 'wb') as f:
	pickle.dump({'tsne':X_2d,'name':filenames}, f)


grid = np.dstack(np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))).reshape(-1, 2)
cost_matrix = cdist(grid, X_2d, "sqeuclidean").astype(np.float32)
out_res = 120
   
cost_matrix = cost_matrix * (100000 / cost_matrix.max())
row_asses, col_asses, _ = lapjv(cost_matrix)
grid_jv = grid[col_asses]

with open('grid_jv.pickle', 'wb') as f:
	pickle.dump({'tsne':X_2d,'name':filenames,'grid_jv':grid_jv}, f)

out = np.ones((10*out_res, 10*out_res, 3))

for i in range(100):
	coordinate_X =  int(np.floor(grid_jv[i,1]*9*out_res))
	coordinate_Y =  int(np.floor(grid_jv[i,0]*9*out_res))

	im=Image.open(filenames[i][1]).convert('RGB').resize((120,120))
	image_data = np.asarray(im)
	out[coordinate_Y:coordinate_Y + out_res, coordinate_X:coordinate_X + out_res]  = image_data
im = Image.fromarray(out.astype(np.uint8))
im.save("tsne_features.png")
plt.imshow(im)